# Chapter 7: การสร้างแอปแชท
## Azure OpenAI API Quickstart


## ภาพรวม
สมุดบันทึกนี้ดัดแปลงมาจาก [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) ซึ่งรวมถึงสมุดบันทึกที่เข้าถึงบริการ [OpenAI](notebook-openai.ipynb) ด้วย

Python OpenAI API ทำงานร่วมกับ Azure OpenAI ได้เช่นกัน โดยมีการปรับเปลี่ยนเล็กน้อย เรียนรู้เพิ่มเติมเกี่ยวกับความแตกต่างได้ที่นี่: [วิธีสลับระหว่าง OpenAI และ Azure OpenAI endpoints ด้วย Python](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)

สำหรับตัวอย่างเริ่มต้นอย่างรวดเร็วเพิ่มเติม โปรดดูที่เอกสารอย่างเป็นทางการ [Azure OpenAI Quickstart Documentation](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio&WT.mc_id=academic-105485-koreyst)


## สารบัญ  

[ภาพรวม](../../../../07-building-chat-applications/python)  
[เริ่มต้นใช้งาน Azure OpenAI Service](../../../../07-building-chat-applications/python)  
[สร้างพรอมต์แรกของคุณ](../../../../07-building-chat-applications/python)  

[กรณีการใช้งาน](../../../../07-building-chat-applications/python)    
[1. สรุปข้อความ](../../../../07-building-chat-applications/python)  
[2. จัดประเภทข้อความ](../../../../07-building-chat-applications/python)  
[3. สร้างชื่อผลิตภัณฑ์ใหม่](../../../../07-building-chat-applications/python)  
[4. ปรับแต่งตัวจัดประเภท](../../../../07-building-chat-applications/python)  
[5. การฝังข้อมูล](../../../../07-building-chat-applications/python)

[แหล่งอ้างอิง](../../../../07-building-chat-applications/python)


### การเริ่มต้นใช้งานกับ Azure OpenAI Service

ลูกค้าใหม่จะต้อง [สมัครขอสิทธิ์เข้าถึง](https://aka.ms/oai/access?WT.mc_id=academic-105485-koreyst) บริการ Azure OpenAI Service  
หลังจากได้รับการอนุมัติเรียบร้อยแล้ว ลูกค้าสามารถเข้าสู่ระบบพอร์ทัล Azure สร้างทรัพยากร Azure OpenAI Service และเริ่มทดลองใช้โมเดลผ่านสตูดิโอได้  

[แหล่งข้อมูลที่ยอดเยี่ยมสำหรับการเริ่มต้นอย่างรวดเร็ว](https://techcommunity.microsoft.com/blog/educatordeveloperblog/azure-openai-service-is-now-generally-available/3719177?WT.mc_id=academic-105485-koreyst)


### สร้างพรอมต์แรกของคุณ  
แบบฝึกหัดสั้นๆ นี้จะให้บทนำพื้นฐานสำหรับการส่งพรอมต์ไปยังโมเดล OpenAI สำหรับงานง่ายๆ "การสรุป"  

**ขั้นตอน**:  
1. ติดตั้งไลบรารี OpenAI ในสภาพแวดล้อม Python ของคุณ  
2. โหลดไลบรารีช่วยเหลือมาตรฐานและตั้งค่าข้อมูลรับรองความปลอดภัย OpenAI ปกติของคุณสำหรับบริการ OpenAI ที่คุณสร้างขึ้น  
3. เลือกโมเดลสำหรับงานของคุณ  
4. สร้างพรอมต์ง่ายๆ สำหรับโมเดล  
5. ส่งคำขอของคุณไปยัง API ของโมเดล!


### 1. ติดตั้ง OpenAI


  > [!หมายเหตุ] ขั้นตอนนี้ไม่จำเป็นหากรันสมุดบันทึกนี้บน Codespaces หรือภายใน Devcontainer


In [ ]:
%pip install openai python-dotenv

### 2. นำเข้าห้องสมุดช่วยเหลือและสร้างข้อมูลประจำตัว


In [ ]:
import os
from openai import AzureOpenAI
import numpy as np
from dotenv import load_dotenv
load_dotenv()

#validate data inside .env file

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

### 3. การค้นหารุ่นที่เหมาะสม  
โมเดล GPT-3.5-turbo หรือ GPT-4 สามารถเข้าใจและสร้างภาษาธรรมชาติได้ บริการนี้มีความสามารถของโมเดลสี่แบบ ซึ่งแต่ละแบบมีระดับพลังและความเร็วที่แตกต่างกัน เหมาะสำหรับงานที่แตกต่างกัน

[Azure OpenAI models](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)


In [ ]:
# Select the General Purpose curie model for text
model = os.environ['AZURE_OPENAI_DEPLOYMENT']

## 4. การออกแบบพรอมต์  

"เวทมนตร์ของโมเดลภาษาขนาดใหญ่คือโดยการฝึกให้ลดข้อผิดพลาดในการทำนายนี้ผ่านปริมาณข้อความจำนวนมาก โมเดลจึงได้เรียนรู้แนวคิดที่เป็นประโยชน์สำหรับการทำนายเหล่านี้ ตัวอย่างเช่น พวกเขาเรียนรู้แนวคิดเช่น"(1):

* วิธีการสะกดคำ
* วิธีการทำงานของไวยากรณ์
* วิธีการเขียนซ้ำความหมาย
* วิธีการตอบคำถาม
* วิธีการสนทนา
* วิธีการเขียนในหลายภาษา
* วิธีการเขียนโค้ด
* เป็นต้น

#### วิธีควบคุมโมเดลภาษาขนาดใหญ่  
"จากอินพุตทั้งหมดของโมเดลภาษาขนาดใหญ่ อินพุตที่มีอิทธิพลมากที่สุดคือพรอมต์ข้อความ(1)

โมเดลภาษาขนาดใหญ่สามารถถูกกระตุ้นให้สร้างผลลัพธ์ได้หลายวิธี:

- คำสั่ง: บอกโมเดลว่าคุณต้องการอะไร
- การเติมข้อความ: กระตุ้นให้โมเดลเติมข้อความในส่วนเริ่มต้นที่คุณต้องการ
- การสาธิต: แสดงให้โมเดลเห็นว่าคุณต้องการอะไร โดยใช้:
- ตัวอย่างไม่กี่ตัวในพรอมต์
- หลายร้อยหรือหลายพันตัวอย่างในชุดข้อมูลฝึกอบรมแบบปรับแต่งละเอียด



#### มีแนวทางพื้นฐานสามข้อในการสร้างพรอมต์:

**แสดงและบอก** ทำให้ชัดเจนว่าคุณต้องการอะไร ไม่ว่าจะผ่านคำสั่ง ตัวอย่าง หรือการผสมผสานของทั้งสองอย่าง หากคุณต้องการให้โมเดลจัดอันดับรายการตามลำดับตัวอักษร หรือจัดประเภทย่อหน้าตามความรู้สึก ให้แสดงให้เห็นว่านั่นคือสิ่งที่คุณต้องการ

**จัดหาข้อมูลคุณภาพ** หากคุณพยายามสร้างตัวจัดประเภทหรือต้องการให้โมเดลทำตามรูปแบบ ให้แน่ใจว่ามีตัวอย่างเพียงพอ ตรวจสอบตัวอย่างของคุณให้ดี — โมเดลมักจะฉลาดพอที่จะมองเห็นข้อผิดพลาดการสะกดคำพื้นฐานและให้คำตอบแก่คุณ แต่ก็อาจสมมติว่านี่เป็นความตั้งใจและอาจส่งผลต่อคำตอบ

**ตรวจสอบการตั้งค่าของคุณ** การตั้งค่า temperature และ top_p ควบคุมว่าระบบมีความแน่นอนในการสร้างคำตอบมากน้อยเพียงใด หากคุณขอคำตอบที่มีคำตอบที่ถูกต้องเพียงคำตอบเดียว คุณควรตั้งค่านี้ให้น้อยลง หากคุณต้องการคำตอบที่หลากหลายมากขึ้น คุณอาจต้องตั้งค่าสูงขึ้น ความผิดพลาดอันดับหนึ่งที่ผู้คนทำกับการตั้งค่าเหล่านี้คือการสมมติว่ามันเป็นการควบคุม "ความฉลาด" หรือ "ความคิดสร้างสรรค์"


แหล่งที่มา: https://learn.microsoft.com/azure/ai-services/openai/overview


ภาพกำลังสร้างคำสั่งข้อความแรกของคุณ!


### 5. ส่ง!


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

### เรียกซ้ำแบบเดียวกัน ผลลัพธ์เปรียบเทียบกันอย่างไร?


In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## สรุปข้อความ  
#### ความท้าทาย  
สรุปข้อความโดยการเพิ่ม 'tl;dr:' ที่ส่วนท้ายของข้อความสั้นๆ สังเกตว่ารูปแบบเข้าใจวิธีการทำงานหลายอย่างโดยไม่ต้องมีคำแนะนำเพิ่มเติม คุณสามารถทดลองใช้คำสั่งที่อธิบายละเอียดกว่าคำว่า tl;dr เพื่อปรับเปลี่ยนพฤติกรรมของโมเดลและปรับแต่งการสรุปที่คุณได้รับ(3)  

งานวิจัยล่าสุดแสดงให้เห็นถึงความก้าวหน้าที่สำคัญในหลายงานและเกณฑ์มาตรฐานของ NLP โดยการฝึกล่วงหน้าบนชุดข้อความขนาดใหญ่ตามด้วยการปรับแต่งเฉพาะงาน แม้ว่าสถาปัตยกรรมจะไม่ขึ้นกับงานโดยตรง แต่วิธีนี้ยังคงต้องการชุดข้อมูลปรับแต่งเฉพาะงานที่มีตัวอย่างนับพันหรือนับหมื่นตัวอย่าง ในทางตรงกันข้าม มนุษย์โดยทั่วไปสามารถทำงานภาษาใหม่ได้จากตัวอย่างเพียงไม่กี่ตัวอย่างหรือคำแนะนำง่ายๆ ซึ่งเป็นสิ่งที่ระบบ NLP ปัจจุบันยังค่อนข้างลำบากที่จะทำได้ ที่นี่เราจะแสดงให้เห็นว่าการขยายขนาดโมเดลภาษาอย่างมากช่วยปรับปรุงประสิทธิภาพแบบไม่ขึ้นกับงานและแบบไม่กี่ตัวอย่างได้ดีขึ้น บางครั้งถึงขั้นแข่งขันได้กับวิธีการปรับแต่งที่ดีที่สุดก่อนหน้านี้  

Tl;dr


# แบบฝึกหัดสำหรับกรณีการใช้งานหลายแบบ  
1. สรุปข้อความ  
2. จัดประเภทข้อความ  
3. สร้างชื่อผลิตภัณฑ์ใหม่  
4. การฝังข้อมูล  
5. ปรับแต่งตัวจัดประเภท


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\ntl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Classify Text  
#### Challenge  
จัดหมวดหมู่รายการตามหมวดหมู่ที่ให้ไว้ในเวลาทำนาย ในตัวอย่างต่อไปนี้ เราให้ทั้งหมวดหมู่และข้อความที่จะจัดหมวดหมู่ในพรอมต์(*playground_reference)  

Customer Inquiry: สวัสดีค่ะ ปุ่มหนึ่งบนแป้นพิมพ์แล็ปท็อปของฉันเพิ่งเสียเมื่อเร็วๆ นี้ และฉันต้องการเปลี่ยนใหม่:  

Classified category:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## สร้างชื่อผลิตภัณฑ์ใหม่  
#### ความท้าทาย  
สร้างชื่อผลิตภัณฑ์จากคำตัวอย่าง ในที่นี้เรารวมข้อมูลเกี่ยวกับผลิตภัณฑ์ที่เราจะสร้างชื่อไว้ในคำสั่ง นอกจากนี้เรายังให้ตัวอย่างที่คล้ายกันเพื่อแสดงรูปแบบที่เราต้องการได้รับ เราได้ตั้งค่าความร้อนสูงเพื่อเพิ่มความสุ่มและการตอบสนองที่สร้างสรรค์มากขึ้น  

คำอธิบายผลิตภัณฑ์: เครื่องทำมิลค์เชคสำหรับใช้ในบ้าน  
คำเมล็ดพันธุ์: เร็ว, สุขภาพดี, กะทัดรัด  
ชื่อผลิตภัณฑ์: HomeShaker, Fit Shaker, QuickShake, Shake Maker  

คำอธิบายผลิตภัณฑ์: รองเท้าคู่หนึ่งที่สามารถพอดีกับขนาดเท้าทุกขนาด  
คำเมล็ดพันธุ์: ปรับตัวได้, พอดี, omni-fit


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

## Embeddings
ส่วนนี้จะแสดงวิธีการดึง embeddings และค้นหาความคล้ายคลึงกันระหว่างคำ ประโยค และเอกสาร เพื่อให้สามารถรันโน้ตบุ๊กต่อไปนี้ได้ คุณจำเป็นต้องปรับใช้โมเดลที่ใช้ `text-embedding-ada-002` เป็นโมเดลฐาน และตั้งชื่อการปรับใช้ในไฟล์ .env โดยใช้ตัวแปร `AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT`


### การจำแนกรูปแบบโมเดล - การเลือกโมเดลฝังตัว

**การจำแนกรูปแบบโมเดล**: {family} - {capability} - {input-type} - {identifier}  

{family}     --> text-embedding  (ตระกูล embeddings)  
{capability} --> ada             (โมเดลฝังตัวอื่นทั้งหมดจะถูกเลิกใช้ในปี 2024)  
{input-type} --> n/a             (ระบุเฉพาะสำหรับโมเดลค้นหา)  
{identifier} --> 002             (เวอร์ชัน 002)  

model = 'text-embedding-ada-002'


  > [!หมายเหตุ] ขั้นตอนต่อไปนี้ไม่จำเป็นหากรันสมุดบันทึกนี้บน Codespaces หรือภายใน Devcontainer


In [ ]:
# Dependencies for embeddings_utils
%pip install matplotlib plotly scikit-learn pandas

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
text = 'the quick brown fox jumped over the lazy dog'
model= os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']
client.embeddings.create(input='[text]', model=model).data[0].embedding

In [ ]:

# compare several words
automobile_embedding  = client.embeddings.create(input='automobile', model=model).data[0].embedding
vehicle_embedding     = client.embeddings.create(input='vehicle', model=model).data[0].embedding
dinosaur_embedding    = client.embeddings.create(input='dinosaur', model=model).data[0].embedding
stick_embedding       = client.embeddings.create(input='stick', model=model).data[0].embedding

print(cosine_similarity(automobile_embedding, vehicle_embedding))
print(cosine_similarity(automobile_embedding, dinosaur_embedding))
print(cosine_similarity(automobile_embedding, stick_embedding))

## การเปรียบเทียบบทความจากชุดข้อมูลข่าวรายวันของ cnn
แหล่งที่มา: https://huggingface.co/datasets/cnn_dailymail


In [ ]:
import pandas as pd
cnn_daily_articles = ['BREMEN, Germany -- Carlos Alberto, who scored in FC Porto\'s Champions League final victory against Monaco in 2004, has joined Bundesliga club Werder Bremen for a club record fee of 7.8 million euros ($10.7 million). Carlos Alberto enjoyed success at FC Porto under Jose Mourinho. "I\'m here to win titles with Werder," the 22-year-old said after his first training session with his new club. "I like Bremen and would only have wanted to come here." Carlos Alberto started his career with Fluminense, and helped them to lift the Campeonato Carioca in 2002. In January 2004 he moved on to FC Porto, who were coached by José Mourinho, and the club won the Portuguese title as well as the Champions League. Early in 2005, he moved to Corinthians, where he impressed as they won the Brasileirão,but in 2006 Corinthians had a poor season and Carlos Alberto found himself at odds with manager, Emerson Leão. Their poor relationship came to a climax at a Copa Sul-Americana game against Club Atlético Lanús, and Carlos Alberto declared that he would not play for Corinthians again while Leão remained as manager. Since January this year he has been on loan with his first club Fluminense. Bundesliga champions VfB Stuttgart said on Sunday that they would sign a loan agreement with Real Zaragoza on Monday for Ewerthon, the third top Brazilian player to join the German league in three days. A VfB spokesman said Ewerthon, who played in the Bundesliga for Borussia Dortmund from 2001 to 2005, was expected to join the club for their pre-season training in Austria on Monday. On Friday, Ailton returned to Germany where he was the league\'s top scorer in 2004, signing a one-year deal with Duisburg on a transfer from Red Star Belgrade. E-mail to a friend .',
                        '(CNN) -- Football superstar, celebrity, fashion icon, multimillion-dollar heartthrob. Now, David Beckham is headed for the Hollywood Hills as he takes his game to U.S. Major League Soccer. CNN looks at how Bekham fulfilled his dream of playing for Manchester United, and his time playing for England. The world\'s famous footballer has begun a five-year contract with the Los Angeles Galaxy team, and on Friday Beckham will meet the press and reveal his new shirt number. This week, we take an in depth look at the life and times of Beckham, as CNN\'s very own "Becks," Becky Anderson, sets out to examine what makes the man tick -- as footballer, fashion icon and global phenomenon. It\'s a long way from the streets of east London to the Hollywood Hills and Becky charts Beckham\'s incredible rise to football stardom, a journey that has seen his skills grace the greatest stages in world soccer. She goes in pursuit of the current hottest property on the sports/celebrity circuit in the U.S. and along the way explores exactly what\'s behind the man with the golden boot. CNN will look back at the life of Beckham, the wonderfully talented youngster who fulfilled his dream of playing for Manchester United, his marriage to pop star Victoria, and the trials and tribulations of playing for England. We\'ll look at the highs (scoring against Greece), the lows (being sent off during the World Cup), the Man. U departure for the Galacticos of Madrid -- and now the Home Depot stadium in L.A. We\'ll ask how Beckham and his family will adapt to life in Los Angeles -- the people, the places to see and be seen and the celebrity endorsement. Beckham is no stranger to exposure. He has teamed with Reggie Bush in an Adidas commercial, is the face of Motorola, is the face on a PlayStation game and doesn\'t need fashion tips as he has his own international clothing line. But what does the star couple need to do to become an accepted part of Tinseltown\'s glitterati? The road to major league football in the U.S.A. is a well-worn route for some of the world\'s greatest players. We talk to some of the former greats who came before him and examine what impact these overseas stars had on U.S. soccer and look at what is different now. We also get a rare glimpse inside the David Beckham academy in L.A, find out what drives the kids and who are their heroes. The perception that in the U.S.A. soccer is a "game for girls" after the teenage years is changing. More and more young kids are choosing the European game over the traditional U.S. sports. E-mail to a friend .',
                        'LOS ANGELES, California (CNN) -- Youssif, the 5-year-old burned Iraqi boy, rounded the corner at Universal Studios when suddenly the little boy hero met his favorite superhero. Youssif has always been a huge Spider-Man fan. Meeting him was "my favorite thing," he said. Spider-Man was right smack dab in front of him, riding a four-wheeler amid a convoy of other superheroes. The legendary climber of buildings and fighter of evil dismounted, walked over to Youssif and introduced himself. Spidey then gave the boy from a far-away land a gentle hug, embracing him in his iconic blue and red tights. He showed Youssif a few tricks, like how to shoot a web from his wrist. Only this time, no web was spun. "All right Youssif!" Spider-Man said after the boy mimicked his wrist movement. Other superheroes crowded around to get a closer look. Even the Green Goblin stopped his villainous ways to tell the boy hi. Youssif remained unfazed. He didn\'t take a liking to Spider-Man\'s nemesis. Spidey was just too cool. "It was my favorite thing," the boy said later. "I want to see him again." He then felt compelled to add: "I know it\'s not the real Spider-Man." This was the day of dreams when the boy\'s nightmares were, at least temporarily, forgotten. He met SpongeBob, Lassie and a 3-year-old orangutan named Archie. The hairy, brownish-red primate took to the boy, grabbing his hand and holding it. Even when Youssif pulled away, Archie would inch his hand back toward the boy\'s and then snatch it. See Youssif enjoy being a boy again » . The boy giggled inside a play area where sponge-like balls shot out of toy guns. It was a far different artillery than what he was used to seeing in central Baghdad, as recently as a week ago. He squealed with delight and raced around the room collecting as many balls as he could. He rode a tram through the back stages at Universal Studios. At one point, the car shook. Fire and smoke filled the air, debris cascaded down and a big rig skidded toward the vehicle. The boy and his family survived the pretend earthquake unscathed. "Even I was scared," the dad said. "Well, I wasn\'t," Youssif replied. The father and mother grinned from ear to ear throughout the day. Youssif pushed his 14-month-old sister, Ayaa, in a stroller. "Did you even need to ask us if we were interested in coming here?" Youssif\'s father said in amazement. "Other than my wedding day, this is the happiest day of my life," he said. Just a day earlier, the mother and father talked about their journey out of Iraq and to the United States. They also discussed that day nine months ago when masked men grabbed their son outside the family home, doused him in gas and set him on fire. His mother heard her boy screaming from inside. The father sought help for his boy across Baghdad, but no one listened. He remembers his son\'s two months of hospitalization. The doctors didn\'t use anesthetics. He could hear his boy\'s piercing screams from the other side of the hospital. Watch Youssif meet his doctor and play with his little sister » . The father knew that speaking to CNN would put his family\'s lives in jeopardy. The possibility of being killed was better than seeing his son suffer, he said. "Anything for Youssif," he said. "We had to do it." They described a life of utter chaos in Baghdad. Neighbors had recently given birth to a baby girl. Shortly afterward, the father was kidnapped and killed. Then, there was the time when some girls wore tanktops and jeans. They were snatched off the street by gunmen. The stories can be even more gruesome. The couple said they had heard reports that a young girl was kidnapped and beheaded --and her killers sewed a dog\'s head on the corpse and delivered it to her family\'s doorstep. "These are just some of the stories," said Youssif\'s mother, Zainab. Under Saddam Hussein, there was more security and stability, they said. There was running water and electricity most of the time. But still life was tough under the dictator, like the time when Zainab\'s uncle disappeared and was never heard from again after he read a "religious book," she said. Sitting in the parking lot of a Target in suburban Los Angeles, Youssif\'s father watched as husbands and wives, boyfriends and girlfriends, parents and their children, came and went. Some held hands. Others smiled and laughed. "Iraq finished," he said in what few English words he knows. He elaborated in Arabic: His homeland won\'t be enjoying such freedoms anytime soon. It\'s just not possible. Too much violence. Too many killings. His two children have only seen war. But this week, the family has seen a much different side of America -- an outpouring of generosity and a peaceful nation at home. "It\'s been a dream," the father said. He used to do a lot of volunteer work back in Baghdad. "Maybe that\'s why I\'m being helped now," the father said. At Universal Studios, he looked out across the valley below. The sun glistened off treetops and buildings. It was a picturesque sight fit for a Hollywood movie. "Good America, good America," he said in English. E-mail to a friend . CNN\'s Arwa Damon contributed to this report.'
]

cnn_daily_article_highlights = ['Werder Bremen pay a club record $10.7 million for Carlos Alberto .\nThe Brazilian midfielder won the Champions League with FC Porto in 2004 .\nSince January he has been on loan with his first club, Fluminense .',
                                'Beckham has agreed to a five-year contract with Los Angeles Galaxy .\nNew contract took effect July 1, 2007 .\nFormer English captain to meet press, unveil new shirt number Friday .\nCNN to look at Beckham as footballer, fashion icon and global phenomenon .',
                                'Boy on meeting Spider-Man: "It was my favorite thing"\nYoussif also met SpongeBob, Lassie and an orangutan at Universal Studios .\nDad: "Other than my wedding day, this is the happiest day of my life"'
]

cnn_df = pd.DataFrame({"articles":cnn_daily_articles, "highligths":cnn_daily_article_highlights})

cnn_df.head()

In [ ]:
article1_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[0], model=model).data[0].embedding
article2_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[1], model=model).data[0].embedding
article3_embedding    = client.embeddings.create(input=cnn_df.articles.iloc[2], model=model).data[0].embedding

print(cosine_similarity(article1_embedding, article2_embedding))
print(cosine_similarity(article1_embedding, article3_embedding))

# References  
- [Azure Documentation - Azure OpenAI Models](https://learn.microsoft.com/azure/cognitive-services/openai/concepts/models?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Examples](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  


# สำหรับความช่วยเหลือเพิ่มเติม  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Contributors
* Louis Li  


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษาอัตโนมัติ [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้ความถูกต้องสูงสุด แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาต้นทางถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลโดยผู้เชี่ยวชาญมนุษย์ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดใด ๆ ที่เกิดจากการใช้การแปลนี้
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
